In [1]:
%%bash
pip install -q -U ../input/kerasapplications/
pip install -q -U /kaggle/input/efficientnet/efficientnet-master

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


# V2 m 512

In [2]:
%%writefile extract_emb_v2_m_512.py

import os, gc, glob
import tensorflow_hub as tfhub
import numpy as np
import pandas as pd
import efficientnet.tfkeras as efn
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
import tensorflow as tf, re, math
from sklearn.model_selection import GroupKFold
import tensorflow.keras.backend as K

IMG_DIM = 512
BATCH_SIZE = 16
EFF_NET_V2 = 'm'
MODEL_PATH = '../input/gl21-models-m-384-p2/m-384-epoch-27.h5'
MODEL_GCS_PATH = f'../input/efficientnetv2-tfhub-weight-files/tfhub_models/efficientnetv2-{EFF_NET_V2}/feature_vector'
N_CLASSES = 203094
HAS_EMBED_DIM = False
PATH_PATTERN_TEST = '../input/landmark-recognition-2021/test/*/*/*/*.jpg'
KNN = 50
TTA = ['original']
KEY = 'm-512'

if len(glob.glob('../input/landmark-recognition-2021/test/*/*/*/*.jpg'))==10345:
    PATH_PATTERN_TEST = '../input/landmark-recognition-2021/test/0/*/*/*.jpg'
    
#  Arcmarginproduct class keras layer
class CurricularFace(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.
    
    A modification of Arcface
    to penalize easy samples
    initially
    
    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(CurricularFace, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(CurricularFace, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)
        self.t = self.add_weight(
            name='t',
            shape=(1),
            initializer=tf.zeros_initializer(),
            dtype='float32',
            trainable=False,
            regularizer=None,
            aggregation=tf.VariableAggregation.MEAN,
            experimental_autocast=False,
            synchronization=tf.VariableSynchronization.ON_READ)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output
    
EFNS = [efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
        efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7]

def freeze_BN(model):
    # Unfreeze layers while leaving BatchNorm layers frozen
    for layer in model.layers:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True
        else:
            layer.trainable = False

def build_model(ef=0):

    margin = CurricularFace(
            n_classes = N_CLASSES,
            s = 30,
            m = 0.5,
            name='head/arc_margin',
            dtype='float32'
            )

    inp = tf.keras.layers.Input(shape = [None, None, 3], name = 'inp1')
    label = tf.keras.layers.Input(shape = (), name = 'inp2')

    out = tfhub.KerasLayer(MODEL_GCS_PATH, trainable=True)(inp)
    x = tf.keras.layers.Dropout(rate=0.1)(out)
    
    x = margin([x, label])
    output = tf.keras.layers.Softmax(dtype='float32')(x)

    model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
    model2 = tf.keras.models.Model(inputs = inp, outputs = out)
    return model, model2

_, model = build_model()

if MODEL_PATH is not None:
    _.load_weights(MODEL_PATH)#, by_name=True)
    
model.summary()

def get_name(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    name = parts[-1]
    return name

def read_images(file_path):
    name = get_name(file_path)
    img = tf.io.read_file(file_path)
    return img,name

def prepare_image(img, tta=None, dim=512):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0
    if tta is not None:
        if 'lr' in tta:
            img = tf.image.flip_left_right(img)
        if 'ud' in tta:
            img = tf.image.flip_up_down(img)
    img = tf.image.resize(img, [dim,dim])
    return img

def get_dataset(pattern, tta=None, batch_size=16, dim=512):
    AUTO = tf.data.experimental.AUTOTUNE
    ds = tf.data.Dataset.list_files(pattern, shuffle=False)
    ds = ds.cache()
    ds = ds.map(read_images, num_parallel_calls=AUTO)
#     ds = ds.map(lambda img, imgname_or_label: (prepare_image(img, tta=tta, dim=dim), imgname_or_label), num_parallel_calls=AUTO)
    ds = ds.map(lambda img, imgname_or_label: prepare_image(img, tta=tta, dim=dim), num_parallel_calls=AUTO)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(AUTO)
    return ds

for tta in TTA:
    test_dataloader = get_dataset(PATH_PATTERN_TEST, tta=None, batch_size=BATCH_SIZE, dim=IMG_DIM)
    test_representations = model.predict(test_dataloader, verbose=1)
    test_files = np.array([i.split('/')[-1].split('.')[0] for i in sorted(glob.glob(PATH_PATTERN_TEST))])
    np.save(f'/tmp/test_{tta}_{KEY}.npy', test_representations)
    
df_tmp = pd.DataFrame()
df_tmp['test_files'] = test_files
df_tmp.to_csv('test_files.csv', index=False)

del test_dataloader, test_representations, model
gc.collect()

Writing extract_emb_v2_m_512.py


In [3]:
!python extract_emb_v2_m_512.py

2021-09-29 21:21:01.583622: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-29 21:21:14.606291: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-29 21:21:14.609652: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-29 21:21:14.651965: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-29 21:21:14.652680: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-09-29 21:21:14.652727: I tensorflow/stream_executor/platform/def

# V1 B6 512

In [4]:
%%writefile extract_emb_v1_b6_512.py

import os, gc, glob
import tensorflow_hub as tfhub
import numpy as np
import pandas as pd
import efficientnet.tfkeras as efn
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
import tensorflow as tf, re, math
from sklearn.model_selection import GroupKFold
import tensorflow.keras.backend as K

IMG_DIM = 512
BATCH_SIZE = 16
EFF_NET_V2 = 'm'
EFF_NET_V1 = 6
MODEL_PATH = '../input/gl21-b6-384-p1/b6-384-epoch-24.h5'
MODEL_GCS_PATH = f'../input/efficientnetv2-tfhub-weight-files/tfhub_models/efficientnetv2-{EFF_NET_V2}/feature_vector'
N_CLASSES = 203094
HAS_EMBED_DIM = False
PATH_PATTERN_TEST = '../input/landmark-recognition-2021/test/*/*/*/*.jpg'
KNN = 50
TTA = ['original']
KEY = 'b6-512' 

if len(glob.glob('../input/landmark-recognition-2021/test/*/*/*/*.jpg'))==10345:
    PATH_PATTERN_TEST = '../input/landmark-recognition-2021/test/0/*/*/*.jpg'
    
#  Arcmarginproduct class keras layer
class CurricularFace(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.
    
    A modification of Arcface
    to penalize easy samples
    initially
    
    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(CurricularFace, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(CurricularFace, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)
        self.t = self.add_weight(
            name='t',
            shape=(1),
            initializer=tf.zeros_initializer(),
            dtype='float32',
            trainable=False,
            regularizer=None,
            aggregation=tf.VariableAggregation.MEAN,
            experimental_autocast=False,
            synchronization=tf.VariableSynchronization.ON_READ)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output
    
EFNS = [efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
        efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7]

def freeze_BN(model):
    # Unfreeze layers while leaving BatchNorm layers frozen
    for layer in model.layers:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True
        else:
            layer.trainable = False

def build_model(ef=0):

    margin = CurricularFace(
            n_classes = N_CLASSES,
            s = 30,
            m = 0.5,
            name='head/arc_margin',
            dtype='float32'
            )

    inp = tf.keras.layers.Input(shape = [None, None, 3], name = 'inp1')
    label = tf.keras.layers.Input(shape = (), name = 'inp2')

    x = EFNS[EFF_NET_V1](weights = None, include_top = False)(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(rate=0.1)(x)
    out = tf.keras.layers.Dense(896)(x)
    
    x = tf.keras.layers.Dropout(rate=0.1)(out)
    
    x = margin([x, label])
    output = tf.keras.layers.Softmax(dtype='float32')(x)

    model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
    model2 = tf.keras.models.Model(inputs = inp, outputs = out)
    return model, model2

_, model = build_model()

if MODEL_PATH is not None:
    _.load_weights(MODEL_PATH)#, by_name=True)
    
model.summary()

def get_name(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    name = parts[-1]
    return name

def read_images(file_path):
    name = get_name(file_path)
    img = tf.io.read_file(file_path)
    return img,name

def prepare_image(img, tta=None, dim=512):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0
    if tta is not None:
        if 'lr' in tta:
            img = tf.image.flip_left_right(img)
        if 'ud' in tta:
            img = tf.image.flip_up_down(img)
    img = tf.image.resize(img, [dim,dim])
    return img

def get_dataset(pattern, tta=None, batch_size=16, dim=512):
    AUTO = tf.data.experimental.AUTOTUNE
    ds = tf.data.Dataset.list_files(pattern, shuffle=False)
    ds = ds.cache()
    ds = ds.map(read_images, num_parallel_calls=AUTO)
#     ds = ds.map(lambda img, imgname_or_label: (prepare_image(img, tta=tta, dim=dim), imgname_or_label), num_parallel_calls=AUTO)
    ds = ds.map(lambda img, imgname_or_label: prepare_image(img, tta=tta, dim=dim), num_parallel_calls=AUTO)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(AUTO)
    return ds

for tta in TTA:
    test_dataloader = get_dataset(PATH_PATTERN_TEST, tta=None, batch_size=BATCH_SIZE, dim=IMG_DIM)
    test_representations = model.predict(test_dataloader, verbose=1)
    test_files = np.array([i.split('/')[-1].split('.')[0] for i in sorted(glob.glob(PATH_PATTERN_TEST))])
    np.save(f'/tmp/test_{tta}_{KEY}.npy', test_representations)
    
df_tmp = pd.DataFrame()
df_tmp['test_files'] = test_files
df_tmp.to_csv('test_files.csv', index=False)

del test_dataloader, test_representations, model
gc.collect()

Writing extract_emb_v1_b6_512.py


In [5]:
!python extract_emb_v1_b6_512.py

2021-09-29 21:22:19.859239: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-29 21:22:24.361632: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-29 21:22:24.362709: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-29 21:22:24.389166: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-29 21:22:24.389875: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-09-29 21:22:24.389939: I tensorflow/stream_executor/platform/def

# Merge Embeddings

In [6]:
%%python

import numpy as np

tta = 'original'
KEYS = ['m-512', 'b6-512']

test_representations = np.concatenate([np.load(f'/tmp/test_{tta}_{key}.npy') for key in KEYS], axis=-1)
np.save(f'/tmp/test_{tta}.npy', test_representations)

# Similarity

In [7]:
import torch
import sys, os
import importlib
from types import SimpleNamespace
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from scipy.special import softmax
from joblib import Parallel, delayed
import seaborn as sns
import scipy as sp

import warnings
warnings.filterwarnings("ignore")

In [8]:
def cos_similarity_matrix(a, b, eps=1e-8):
    a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
    a_norm = a / torch.max(a_n, eps * torch.ones_like(a_n))
    b_norm = b / torch.max(b_n, eps * torch.ones_like(b_n))
    sim_mt = torch.mm(a_norm, b_norm.transpose(0, 1))
    return sim_mt

def get_topk_cossim(test_emb, tr_emb, batchsize = 64, k=10, device='cuda:0',verbose=True):
    tr_emb = torch.tensor(tr_emb, dtype = torch.float32, device=torch.device(device))
    test_emb = torch.tensor(test_emb, dtype = torch.float32, device=torch.device(device))
    vals = []
    inds = []
    for test_batch in tqdm(test_emb.split(batchsize),disable=1-verbose):
        sim_mat = cos_similarity_matrix(test_batch, tr_emb)
        vals_batch, inds_batch = torch.topk(sim_mat, k=k, dim=1)
        vals += [vals_batch.detach().cpu()]
        inds += [inds_batch.detach().cpu()]
    vals = torch.cat(vals)
    inds = torch.cat(inds)
    
    del tr_emb, test_emb, test_batch, sim_mat, vals_batch, inds_batch
    gc.collect()
    
    return vals, inds

def get_topk_cossim_sub(test_emb, tr_emb, vals_x, batchsize = 64, k=10, device='cuda:0',verbose=True):
    tr_emb = torch.tensor(tr_emb, dtype = torch.float32, device=torch.device(device))
    test_emb = torch.tensor(test_emb, dtype = torch.float32, device=torch.device(device))
    vals_x = torch.tensor(vals_x, dtype = torch.float32, device=torch.device(device))
    vals = []
    inds = []
    for test_batch in tqdm(test_emb.split(batchsize),disable=1-verbose):
        sim_mat = cos_similarity_matrix(test_batch, tr_emb)
        sim_mat = torch.clamp(sim_mat,0,1) - vals_x.repeat(sim_mat.shape[0], 1)
        
        vals_batch, inds_batch = torch.topk(sim_mat, k=k, dim=1)
        vals += [vals_batch.detach().cpu()]
        inds += [inds_batch.detach().cpu()]
    vals = torch.cat(vals)
    inds = torch.cat(inds)
    
    del test_emb, tr_emb, vals_x, sim_mat, vals_batch, inds_batch, test_batch
    gc.collect()
    
    return vals, inds

In [9]:
import gc
import pandas as pd
from sklearn.preprocessing import normalize

df1 = pd.read_csv('../input/gl21-test-order/test_image_2019.csv').rename(columns={'img_name':'id'})
df1['id'] = df1['id'].apply(lambda x: x.split('.')[0])

df2 = pd.read_csv('../input/gl21-additional-train-images-metadata/recognition_solution_v2.1.csv')[['id', 'landmarks']]
df1 = df1.merge(df2, how='left', on=['id']).fillna(-1)

def fn(x):
    try:
        return int(x.split()[0])
    except:
        return -1

df1['landmarks'] = df1.landmarks.apply(fn)

landmark_ids = set(pd.read_csv('../input/landmark-recognition-2021/train.csv').landmark_id.unique())
# landmark_ids = set(list(range(1,81315)))
landmark_ids.add(-1)

targets = df1.landmarks.values

del df1, df2
gc.collect()

0

In [10]:
# %%time

import os, glob, gc
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer
from sklearn.random_projection import GaussianRandomProjection

embeddings = ['../input/gl-2021-m-384-epoch-16-5-val', '../input/gl2021-b6-384']

# val_embeddings = np.concatenate([np.load(os.path.join(folder, f'val_emb.npy'))[np.argsort(np.load(os.path.join(folder, f'val_indx.npy')))] for folder in embeddings])
val_embeddings = []

for i in range(20):
    val_embeddings_tmp = np.concatenate([np.load(os.path.join(folder, f'val_emb_{i}.npy'))[np.argsort(np.load(os.path.join(folder, f'val_indx_{i}.npy')))] for folder in embeddings], axis=-1)
    val_embeddings.append(val_embeddings_tmp)
    del val_embeddings_tmp
    gc.collect()
    
val_embeddings = np.concatenate(val_embeddings, axis=0)
print(val_embeddings.shape)

val_embeddings = val_embeddings[[i in landmark_ids for i in targets]]
targets = targets[[i in landmark_ids for i in targets]]

val_embeddings = normalize(val_embeddings, norm="l2")

nonlandmark_embeddings = val_embeddings[targets==-1]
val_embeddings = np.load(f'/tmp/test_original.npy')


f = QuantileTransformer(output_distribution="normal")

f.fit(np.concatenate([val_embeddings],axis=0))

val_embeddings = f.transform(val_embeddings)
nonlandmark_embeddings = f.transform(nonlandmark_embeddings)

vals_blend = []

labels_blend = []
inds_blend = []

KNN = 3

##############################################
if len(glob.glob('../input/landmark-recognition-2021/test/*/*/*/*.jpg'))==10345:
    N = 2
else:
    N = 400

embeddings = ['../input/gl-2021-m-384-epoch-16-5-train', '../input/gl2021-b6-384']
with torch.no_grad():
    for indx in tqdm(range(N)):

        tr_embeddings = np.concatenate([np.load(os.path.join(folder, f'train_emb_{indx}.npy'))[np.argsort(np.load(os.path.join(folder, f'train_indx_{indx}.npy')))] for folder in embeddings], axis=-1)
        targets_train = np.load(os.path.join(embeddings[0], f'train_ids_{indx}.npy'))[np.argsort(np.load(os.path.join(embeddings[0], f'train_indx_{indx}.npy')))]
        tr_embeddings = tr_embeddings[[i in landmark_ids for i in targets_train]]
        targets_train = targets_train[[i in landmark_ids for i in targets_train]]

        tr_embeddings = normalize(tr_embeddings, norm="l2")

        tr_embeddings = f.transform(tr_embeddings)

        print(tr_embeddings.shape, targets_train.shape, val_embeddings.shape, targets.shape, nonlandmark_embeddings.shape)

        vals_nl, inds_nl = get_topk_cossim(tr_embeddings, nonlandmark_embeddings, k=5)
        vals_nl = vals_nl[:,:].mean(axis=1).detach().cpu().numpy()

        vals, inds = get_topk_cossim_sub(val_embeddings, tr_embeddings, vals_nl, k=KNN)
        vals = vals.data.cpu().numpy()
        inds = inds.data.cpu().numpy()
        labels = np.concatenate([targets_train[inds[:,i]].reshape(-1,1) for i in range(inds.shape[1])], axis=1)

        vals_blend.append(vals)
        labels_blend.append(labels)
        inds_blend.append(inds)
        
        del tr_embeddings, targets_train, vals_nl, inds_nl, vals, inds, labels
        gc.collect()
        
        torch.cuda.empty_cache()

vals = np.concatenate(vals_blend, axis=1)
inds = np.concatenate(inds_blend, axis=1)
labels = np.concatenate(labels_blend, axis=1)

args = np.argsort(vals, axis=-1)

for i in range(len(vals)):
    vals[i,-KNN:] = vals[i, args[i, -KNN:]]
    inds[i,-KNN:] = inds[i, args[i, -KNN:]]
    labels[i,-KNN:] = labels[i, args[i, -KNN:]]
    
vals = vals[:, -KNN:]
inds = inds[:, -KNN:]
labels = labels[:, -KNN:]

(117577, 2176)


  0%|          | 0/2 [00:00<?, ?it/s]

(7925, 2176) (7925,) (651, 2176) (117172,) (115605, 2176)


  0%|          | 0/124 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

(7935, 2176) (7935,) (651, 2176) (117172,) (115605, 2176)


  0%|          | 0/124 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [11]:
# vals = np.concatenate(vals_blend, axis=1)
# inds = np.concatenate(inds_blend, axis=1)
# labels = np.concatenate(labels_blend, axis=1)

In [12]:
# vals.shape

In [13]:
def global_average_precision_score(y_true, y_pred, ignore_non_landmarks=False):
    indexes = np.argsort(y_pred[1])[::-1]
    queries_with_target = (y_true < 210_000).sum()
    correct_predictions = 0
    total_score = 0.
    i = 1
    for k in indexes:
        if ignore_non_landmarks and y_true[k] == -1:
            continue
        if y_pred[0][k] == -1:
            continue
        relevance_of_prediction_i = 0
        if y_true[k] == y_pred[0][k]:
            correct_predictions += 1
            relevance_of_prediction_i = 1
        precision_at_rank_i = correct_predictions / i
        total_score += precision_at_rank_i * relevance_of_prediction_i
        i += 1
    return 1 / queries_with_target * total_score

def comp_metric(y_true, logits, ignore_non_landmarks=False):
    
    score = global_average_precision_score(y_true, logits, ignore_non_landmarks=ignore_non_landmarks)
    return score

In [14]:
from collections import defaultdict

vals_new = []
labels_new = []

for i in tqdm(range(len(vals))):
    cnts = defaultdict(list)

    x = 0
    for j,l in enumerate(labels[i,:]):

        curr = vals[i][j]

        cnts[l].append(curr)

    for k,v in cnts.items():
        cnts[k] = np.sum(v)
        
    labels_new.append(max(cnts, key=cnts.get))
    vals_new.append(cnts[labels_new[-1]])
        
l = np.array(labels_new).reshape(-1)
v = np.array(vals_new).reshape(-1)

# val_score = comp_metric(targets, [l, v], ignore_non_landmarks=False)
# print(val_score)
# val_score = comp_metric(targets, [l, v], ignore_non_landmarks=True)
# print(val_score)

  0%|          | 0/651 [00:00<?, ?it/s]

In [15]:
vals_2, inds_2 = get_topk_cossim(val_embeddings, nonlandmark_embeddings, k=11)
# starting from index 1 on val as index 0 is the same image
vals_2 = vals_2[:,1:].mean(axis=1).detach().cpu().numpy()

  0%|          | 0/11 [00:00<?, ?it/s]

In [16]:
# penalize by C
import scipy as sp

l3 = pd.Series(l.copy()).reset_index(drop=True)
v3 = v.copy()

v3 -= 1*vals_2

# val_score = comp_metric(targets[:], [l3[:], v3[:]], ignore_non_landmarks=False)
# print(val_score)
# val_score = comp_metric(targets[:], [l3[:], v3[:]], ignore_non_landmarks=True)
# print(val_score)

In [17]:
df = pd.read_csv('./test_files.csv').rename(columns={'test_files':'id'})

In [18]:
l.shape, v.shape, len(vals), vals_2.shape, inds_2.shape, l3.shape, v3.shape, df.shape

((651,), (651,), 651, (651,), torch.Size([651, 11]), (651,), (651,), (651, 1))

In [19]:
df['landmarks'] = [f'{landmark_id} {confidence}' for (landmark_id, confidence) in zip(l3, v3)]
df.to_csv('submission.csv', index=False)